# Data Quality Rule Recommendation Engine - Usage Examples

This notebook demonstrates how to use the Data Quality Rule Recommendation Engine to analyze datasets and generate intelligent data quality rules using LLM inference.

## Contents
1. Setup and Installation
2. Basic Usage
3. Advanced Features
4. Custom Rule Generation
5. Testing and Validation

## 1. Setup and Installation

First, let's import the required libraries and initialize the recommendation engine.

In [ ]:
import pandas as pd
import json
from backend.recommendation_engine import DataQualityRuleRecommendationEngine
from backend.agents.rule_recommender import DataQualityRuleRecommender

# Initialize the recommendation engine
engine = DataQualityRuleRecommendationEngine()

## 2. Basic Usage

Let's load a sample dataset and generate basic data quality recommendations.

In [ ]:
# Load sample data
df = pd.read_csv('sample_customer_data.csv')

# Display basic dataset information
print("Dataset Overview:")
print("-" * 50)
print(f"Number of rows: {len(df)}")
print(f"Number of columns: {len(df.columns)}")
print("\nColumns:")
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

In [ ]:
# Generate recommendations
recommendations = engine.analyze_data_and_recommend_rules(
    df=df,
    technical_metadata=None,
    data_lineage=None
)

# Display recommendations summary
severity_counts = {
    'CRITICAL': 0,
    'HIGH': 0,
    'MEDIUM': 0,
    'LOW': 0
}

for rec in recommendations:
    severity_counts[rec['severity']] += 1

print("Recommendations Summary:")
print("-" * 50)
print(f"Total rules: {len(recommendations)}")
for severity, count in severity_counts.items():
    print(f"{severity}: {count} rules")

# Display detailed recommendations
print("\nDetailed Recommendations:")
print("-" * 50)
for i, rec in enumerate(recommendations[:5], 1):
    print(f"\nRule {i}:")
    print(f"Name: {rec['rule_name']}")
    print(f"Type: {rec['rule_type']}")
    print(f"Severity: {rec['severity']}")
    print(f"Description: {rec['description']}")
    print(f"SQL: {rec['sql_rule']}")

## 3. Advanced Features

Now let's explore some advanced features, including custom rule generation and pattern detection.

In [ ]:
# Initialize the rule recommender for advanced features
recommender = DataQualityRuleRecommender()

# Select a specific column for detailed analysis
column_name = 'customer_email'  # Replace with an actual column name from your dataset

# Load metadata files
with open('business_glossary.json', 'r') as f:
    business_glossary = json.load(f)
    
with open('technical_metadata.json', 'r') as f:
    technical_metadata = json.load(f)
    
with open('data_lineage.json', 'r') as f:
    data_lineage = json.load(f)

# Generate custom rules for the column
custom_rules = recommender.analyze_and_recommend(
    df=df,
    column_name=column_name,
    business_glossary=business_glossary,
    technical_metadata=technical_metadata,
    data_lineage=data_lineage
)

# Display custom rules
print(f"Custom Rules for column '{column_name}':")
print("-" * 50)
for i, rule in enumerate(custom_rules, 1):
    print(f"\nCustom Rule {i}:")
    print(f"Name: {rule['rule_name']}")
    print(f"Description: {rule['description']}")
    print(f"SQL: {rule['sql_rule']}")
    if 'spark_sql' in rule:
        print(f"Spark SQL: {rule['spark_sql']}")

## 4. Pattern Detection

Let's analyze patterns in the data using the built-in pattern detection capabilities.

In [ ]:
def analyze_column_patterns(df, column_name):
    """Analyze patterns in a specific column"""
    series = df[column_name]
    patterns = {}
    
    if pd.api.types.is_numeric_dtype(series):
        # Numerical column analysis
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = series[(series < lower_bound) | (series > upper_bound)]
        
        patterns['statistics'] = {
            'mean': series.mean(),
            'std': series.std(),
            'min': series.min(),
            'max': series.max(),
            'median': series.median()
        }
        patterns['outliers'] = {
            'count': len(outliers),
            'percentage': (len(outliers) / len(series)) * 100,
            'bounds': {'lower': float(lower_bound), 'upper': float(upper_bound)}
        }
    else:
        # String column analysis
        patterns['length_stats'] = {
            'avg_length': series.str.len().mean(),
            'max_length': series.str.len().max(),
            'min_length': series.str.len().min()
        }
        patterns['unique_values'] = {
            'count': series.nunique(),
            'percentage': (series.nunique() / len(series)) * 100
        }
        patterns['top_values'] = series.value_counts().head(5).to_dict()
    
    return patterns

# Analyze patterns for a few columns
for col in df.columns[:3]:  # Analyze first 3 columns
    print(f"\nPattern Analysis for '{col}':")
    print("-" * 50)
    patterns = analyze_column_patterns(df, col)
    print(json.dumps(patterns, indent=2))

## 5. Testing and Validation

Finally, let's validate the recommendations and test their effectiveness.

In [ ]:
def validate_rules(df, recommendations):
    """Validate rules against the dataset"""
    results = []
    
    for rule in recommendations:
        try:
            # Convert SQL rule to a pandas query where possible
            if 'IS NULL' in rule['sql_rule']:
                violation_count = df[rule['column']].isnull().sum()
            elif 'DISTINCT' in rule['sql_rule']:
                violation_count = len(df) - df[rule['column']].nunique()
            else:
                # For more complex rules, just report them
                violation_count = None
            
            results.append({
                'rule_name': rule['rule_name'],
                'severity': rule['severity'],
                'violations': violation_count,
                'status': 'Validated' if violation_count is not None else 'Complex Rule'
            })
        except Exception as e:
            results.append({
                'rule_name': rule['rule_name'],
                'severity': rule['severity'],
                'violations': None,
                'status': f'Error: {str(e)}'
            })
    
    return results

# Validate the recommendations
validation_results = validate_rules(df, recommendations)

# Display validation results
print("Rule Validation Results:")
print("-" * 50)
for result in validation_results:
    print(f"\nRule: {result['rule_name']}")
    print(f"Severity: {result['severity']}")
    print(f"Violations: {result['violations']}")
    print(f"Status: {result['status']}")

## Conclusion

This notebook demonstrated the key features of the Data Quality Rule Recommendation Engine:
1. Basic rule generation using LLM inference
2. Custom rule creation based on data patterns
3. Advanced pattern detection and analysis
4. Rule validation and testing

For more information, refer to the documentation in the repository.